In [ ]:
import sys
import rospy
import moveit_commander
from moveit_msgs.srv import GetJointWithConstraints, GetJointWithConstraintsRequest
from moveit_msgs.msg import RobotState, Constraints, OrientationConstraint, MoveItErrorCodes, SamplingDistribution
from geometry_msgs.msg import Quaternion, Pose
import numpy as np

In [ ]:
moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('constrained_motion_planning_test', anonymous=True)

In [ ]:
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
move_group = moveit_commander.MoveGroupCommander("arm")
joint_names = move_group.get_active_joints()
sample_joint_with_constraints_service = rospy.ServiceProxy('/sample_joint_with_constraints', GetJointWithConstraints)

In [ ]:
move_group.set_planner_id('CDISTRIBUTIONRRTConfigDefault')

In [ ]:
# initial a constraint
horizontal_constraint = Constraints()
horizontal_constraint.name = "use_equality_constraints"

oc = OrientationConstraint()

oc.parameterization = OrientationConstraint.ROTATION_VECTOR;
oc.header.frame_id = "base_link";
oc.header.stamp = rospy.Time(0)
oc.link_name = "wrist_roll_link";
constrained_quaternion = Quaternion();
constrained_quaternion.x = 0.0
constrained_quaternion.y = 0.0
constrained_quaternion.z = 0.0
constrained_quaternion.w = 1.0
oc.orientation = constrained_quaternion
oc.weight = 1.0

oc.absolute_x_axis_tolerance = 0.1
oc.absolute_y_axis_tolerance = 0.1
oc.absolute_z_axis_tolerance = 2 * 3.1415
horizontal_constraint.orientation_constraints.append(oc)

# need to set in-hand pose
in_hand_pose = Pose()
in_hand_pose.position.x = 0.0
in_hand_pose.position.y = 0.0
in_hand_pose.position.z = 0.0
in_hand_pose.orientation.x = 0.0
in_hand_pose.orientation.y = 0.0
in_hand_pose.orientation.z = 0.0
in_hand_pose.orientation.w = 1.0
horizontal_constraint.in_hand_pose = in_hand_pose

In [ ]:
# sample start and goal configuration satisfying constraint
sample_request = GetJointWithConstraintsRequest()
sample_request.constraints = horizontal_constraint
sample_request.group_name = "arm"
init_sample = sample_joint_with_constraints_service(sample_request)
goal_sample = sample_joint_with_constraints_service(sample_request)
print("error code of init sample ", init_sample.error_code)
print("error code of target sample ", goal_sample.error_code)

In [ ]:
def getProperJointState(all_name, all_joint, selected_name):
    result = []
    for sn in selected_name:
        result.append(all_joint[all_name.index(sn)])
    return result

In [ ]:
# convert them into joint
init_joint = getProperJointState(init_sample.solution.joint_state.name, init_sample.solution.joint_state.position, joint_names)
goal_joint = getProperJointState(goal_sample.solution.joint_state.name, goal_sample.solution.joint_state.position, joint_names)

In [ ]:
# set start and goal arm configurations
moveit_robot_state = RobotState()
moveit_robot_state.joint_state.name = joint_names
moveit_robot_state.joint_state.position = init_joint

move_group.set_start_state(moveit_robot_state)
move_group.set_joint_value_target(goal_joint)

# set distribution
distribution_sequence = []

mean = np.zeros(7)
covariance = np.eye(7)

distribution = SamplingDistribution()
distribution.distribution_mean = mean.tolist()
distribution.distribution_convariance = covariance.flatten().tolist()
distribution_sequence.append(distribution)

move_group.set_distribution(distribution_sequence)
# clean the motion planner data(if you use some prm like motion planner)
move_group.set_clean_planning_context_flag(True)
# you must set path constraint for CDistribution rrt, or it will cause error in moveit.
move_group.set_path_constraints(horizontal_constraint)
move_group.set_in_hand_pose(in_hand_pose)
result = move_group.plan()

# you must clear them for next planning.
move_group.clear_in_hand_pose()
move_group.clear_path_constraints()
move_group.clear_distribution()